In [12]:
import logging
import warnings


# Wine Quality Sample
def train(it_n_estimators, it_learning_rate, it_max_depth):
    import pandas as pd
    import numpy as np
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    from xgboost import XGBRegressor

    import mlflow
    import mlflow.sklearn
    import mlflow.xgboost
    from mlflow.models import infer_signature

    logging.basicConfig(level=logging.WARN)
    logger = logging.getLogger(__name__)

    def eval_metrics(actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    warnings.filterwarnings("ignore")
    np.random.seed(40)


    # Read the wine-quality csv file from the URL
    csv_url = (
        "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
    )
    try:
        data = pd.read_csv(csv_url, sep=";")
    except Exception as e:
        logger.exception(
            f"Unable to download training & test CSV, check your internet connection. Error: {e}"
        )


    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data, test_size=0.25, random_state=40)


    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]

    # Set default values if no n_estimators is provided
    n_estimators = 100 if int(it_n_estimators) is None else int(it_n_estimators)

    # Set default values if no learning_rate is provided
    learning_rate = 0.1 if float(it_learning_rate) is None else float(it_learning_rate)

    # Set default values if no max_depht is provided
    max_depth = 5 if int(it_max_depth) is None else int(it_max_depth)

    # Useful for multiple runs (only doing one run in this sample notebook)
    with mlflow.start_run():
        # Execute ElasticNet
        lr = XGBRegressor(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            random_state=40
        )
        lr.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        # Print out metrics
        print(f"XGBoost Regressor (n_estimators={n_estimators:f}, learning_rate={learning_rate:f}, max_depth={max_depth:f}):")
        print(f"  RMSE: {rmse}")
        print(f"  MAE: {mae}")
        print(f"  R2: {r2}")

        # Infer model signature
        predictions = lr.predict(train_x)
        signature = infer_signature(train_x, predictions)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("learning_rate", learning_rate)
        mlflow.log_param("max_depth", max_depth)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        mlflow.log_param("data_set", "wine_quality_v.1.0.0")

        mlflow.sklearn.log_model(lr, "model", signature=signature)

        mlflow.xgboost.log_model(lr, "xgboost_model", registered_model_name="xgboost_v.1.0.0_dataset_v.1.0.0")

        

train(200, 0.2, 10)

In [13]:
train(150, 0.01, 2)

XGBoost Regressor (n_estimators=150.000000, learning_rate=0.010000, max_depth=2.000000):
  RMSE: 0.6884537145850858
  MAE: 0.5528584349155427
  R2: 0.32844246803017596


Successfully registered model 'xgboost_v.1.0.0_dataset_v.1.0.0'.
Created version '1' of model 'xgboost_v.1.0.0_dataset_v.1.0.0'.


In [14]:
train(200, 0.2, 10)

XGBoost Regressor (n_estimators=200.000000, learning_rate=0.200000, max_depth=10.000000):
  RMSE: 0.6111864460712434
  MAE: 0.39593769192695616
  R2: 0.4707252709982688


Registered model 'xgboost_v.1.0.0_dataset_v.1.0.0' already exists. Creating a new version of this model...
Created version '2' of model 'xgboost_v.1.0.0_dataset_v.1.0.0'.


In [15]:
train(100, 0.1, 5)

XGBoost Regressor (n_estimators=100.000000, learning_rate=0.100000, max_depth=5.000000):
  RMSE: 0.6040791510588813
  MAE: 0.44500560402870176
  R2: 0.4829632379384101


Registered model 'xgboost_v.1.0.0_dataset_v.1.0.0' already exists. Creating a new version of this model...
Created version '3' of model 'xgboost_v.1.0.0_dataset_v.1.0.0'.
